In [1]:
#!pip install pyspark
#!pip install findspark

In [2]:
import findspark

In [3]:
findspark.init("C:/spark/spark-3.3.2-bin-hadoop2")

In [4]:
import pyspark

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
color = sns.color_palette()
%matplotlib inline
pd.options.mode.chained_assignment = None  # default='warn'
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [6]:
import findspark
findspark.init

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.ml.feature import StringIndexer
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.functions import *
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count
from pyspark.sql.window import Window
from pyspark.sql import functions as F
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

In [7]:
spark = SparkSession.Builder().appName('Project').getOrCreate()
sqlCtx = SQLContext(spark)

C:\spark/spark-3.3.2-bin-hadoop2\python\pyspark\sql\context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [8]:
# Importing the data :
aisles = spark.read.csv("aisles.csv", header='true', inferSchema='true')
departments = spark.read.csv("departments.csv", header='true', inferSchema='true')
orders = spark.read.csv("orders.csv", header='true', inferSchema='true')
products = spark.read.csv("products.csv", header='true', inferSchema='true')
order_products_prior= spark.read.csv("order_products__prior.csv", header='true', inferSchema='true')

In [9]:
merged = order_products_prior.join(orders, on="order_id",how="left")
merged = merged.join(products,on="product_id",how="left")

In [10]:
merged.show(5)

+----------+--------+-----------------+---------+-------+--------+------------+---------+-----------------+----------------------+--------------------+--------+-------------+
|product_id|order_id|add_to_cart_order|reordered|user_id|eval_set|order_number|order_dow|order_hour_of_day|days_since_prior_order|        product_name|aisle_id|department_id|
+----------+--------+-----------------+---------+-------+--------+------------+---------+-----------------+----------------------+--------------------+--------+-------------+
|     47546| 3163494|                1|        1| 162382|   prior|          19|        1|               12|                   3.0|Chocolate Coconut...|      91|           16|
|     19019| 3163494|                2|        1| 162382|   prior|          19|        1|               12|                   3.0|Uncured Slow Cook...|      96|           20|
|     21162| 3163494|                3|        0| 162382|   prior|          19|        1|               12|                  

In [11]:
for i in merged.columns:
    print(merged[i])

Column<'product_id'>
Column<'order_id'>
Column<'add_to_cart_order'>
Column<'reordered'>
Column<'user_id'>
Column<'eval_set'>
Column<'order_number'>
Column<'order_dow'>
Column<'order_hour_of_day'>
Column<'days_since_prior_order'>
Column<'product_name'>
Column<'aisle_id'>
Column<'department_id'>


In [12]:
from pyspark.sql.types import StructType,StringType,IntegerType,DoubleType,LongType

In [13]:
schema1 = StructType() \
    .add("product_id", IntegerType(), True) \
    .add("order_id", IntegerType(), True) \
    .add("add_to_cart_order", IntegerType(), True) \
    .add("reordered", IntegerType(), True) \
    .add("user_id", IntegerType(), True) \
    .add("eval_set", StringType(), True) \
    .add("order_number", IntegerType(), True) \
    .add("order_dow", IntegerType(), True) \
    .add("order_hour_of_day", IntegerType(), True) \
    .add("days_since_prior_order", DoubleType(), True) \
    .add("product_name", StringType(), True) \
    .add("aisle_id", IntegerType(), True) \
    .add("department_id", IntegerType(), True)

In [14]:
import os
os.environ["hadoop.home.dir"] = "C:/hadoop"

In [15]:
merged.write.csv("C:/Users/PRASAD/Desktop/Main_Project/merged.csv", header=True, mode="overwrite")

In [16]:
df_merged = spark.read.format("csv").option("header","False").schema(schema1).load("C:/Users/PRASAD/Desktop/Main_Project/merged.csv")

In [17]:
df_merged.count()

32434502

In [18]:
df_merged.registerTempTable("merged_table")

C:\spark/spark-3.3.2-bin-hadoop2\python\pyspark\sql\dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [22]:
df_merged.columns

['product_id',
 'order_id',
 'add_to_cart_order',
 'reordered',
 'user_id',
 'eval_set',
 'order_number',
 'order_dow',
 'order_hour_of_day',
 'days_since_prior_order',
 'product_name',
 'aisle_id',
 'department_id']

In [29]:
# Most sold Product
product_counts = df_merged.groupBy("product_name"
                                  ).agg({"product_id": "count"}
                                        ).withColumnRenamed("count(product_id)", "Frq_sold"
                                                           ).orderBy(col("Frq_sold").desc())

product_counts.show(10)

+--------------------+--------+
|        product_name|Frq_sold|
+--------------------+--------+
|              Banana|  472565|
|Bag of Organic Ba...|  379450|
|Organic Strawberries|  264683|
|Organic Baby Spinach|  241921|
|Organic Hass Avocado|  213584|
|     Organic Avocado|  176815|
|         Large Lemon|  152657|
|        Strawberries|  142951|
|               Limes|  140627|
|  Organic Whole Milk|  137905|
+--------------------+--------+
only showing top 10 rows



In [32]:
product_counts_sql = spark.sql("select product_name,count(product_id) Frq_sold from merged_table group by product_name order by Frq_sold desc;")
product_counts_sql.show(10)

+--------------------+--------+
|        product_name|Frq_sold|
+--------------------+--------+
|              Banana|  472565|
|Bag of Organic Ba...|  379450|
|Organic Strawberries|  264683|
|Organic Baby Spinach|  241921|
|Organic Hass Avocado|  213584|
|     Organic Avocado|  176815|
|         Large Lemon|  152657|
|        Strawberries|  142951|
|               Limes|  140627|
|  Organic Whole Milk|  137905|
+--------------------+--------+
only showing top 10 rows



In [34]:
filtered_products = product_counts.filter(col("Frq_sold") > 100000)
print(filtered_products.count())
filtered_products.show()

15
+--------------------+--------+
|        product_name|Frq_sold|
+--------------------+--------+
|              Banana|  472565|
|Bag of Organic Ba...|  379450|
|Organic Strawberries|  264683|
|Organic Baby Spinach|  241921|
|Organic Hass Avocado|  213584|
|     Organic Avocado|  176815|
|         Large Lemon|  152657|
|        Strawberries|  142951|
|               Limes|  140627|
|  Organic Whole Milk|  137905|
| Organic Raspberries|  137057|
|Organic Yellow Onion|  113426|
|      Organic Garlic|  109778|
|    Organic Zucchini|  104823|
| Organic Blueberries|  100060|
+--------------------+--------+



In [37]:
filtered_products_sql = spark.sql(
    "select * from (select product_name,count(product_id) Frq_sold from merged_table group by product_name order by Frq_sold desc) where Frq_sold >100000;")

print(filtered_products_sql.count())
filtered_products.show()

15
+--------------------+--------+
|        product_name|Frq_sold|
+--------------------+--------+
|              Banana|  472565|
|Bag of Organic Ba...|  379450|
|Organic Strawberries|  264683|
|Organic Baby Spinach|  241921|
|Organic Hass Avocado|  213584|
|     Organic Avocado|  176815|
|         Large Lemon|  152657|
|        Strawberries|  142951|
|               Limes|  140627|
|  Organic Whole Milk|  137905|
| Organic Raspberries|  137057|
|Organic Yellow Onion|  113426|
|      Organic Garlic|  109778|
|    Organic Zucchini|  104823|
| Organic Blueberries|  100060|
+--------------------+--------+



In [50]:
df_merged = df_merged.join(filtered_products, on="product_name")

In [51]:
df_merged.show(5)

+--------------------+----------+--------+-----------------+---------+-------+--------+------------+---------+-----------------+----------------------+--------+-------------+--------+
|        product_name|product_id|order_id|add_to_cart_order|reordered|user_id|eval_set|order_number|order_dow|order_hour_of_day|days_since_prior_order|aisle_id|department_id|Frq_sold|
+--------------------+----------+--------+-----------------+---------+-------+--------+------------+---------+-----------------+----------------------+--------+-------------+--------+
|Organic Hass Avocado|     47209|      28|               12|        1|  98256|   prior|          29|        3|               13|                   6.0|      24|            4|  213584|
|Organic Hass Avocado|     47209| 2378024|                9|        1| 188532|   prior|          25|        0|               15|                   1.0|      24|            4|  213584|
|Organic Hass Avocado|     47209|      53|                3|        1|  87889|  

In [52]:
df_merged.count()

2888302

In [99]:
df = df_merged.toPandas()
df.info()

C:\spark/spark-3.3.2-bin-hadoop2\python\pyspark\sql\pandas\utils.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(pandas.__version__) < LooseVersion(minimum_pandas_version):
C:\spark/spark-3.3.2-bin-hadoop2\python\pyspark\sql\pandas\utils.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(pandas.__version__) < LooseVersion(minimum_pandas_version):


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2888302 entries, 0 to 2888301
Data columns (total 14 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   product_name            object 
 1   product_id              int32  
 2   order_id                int32  
 3   add_to_cart_order       int32  
 4   reordered               int32  
 5   user_id                 int32  
 6   eval_set                object 
 7   order_number            int32  
 8   order_dow               int32  
 9   order_hour_of_day       int32  
 10  days_since_prior_order  float64
 11  aisle_id                int32  
 12  department_id           int32  
 13  Frq_sold                int64  
dtypes: float64(1), int32(10), int64(1), object(2)
memory usage: 198.3+ MB


In [54]:
print(df_merged.printSchema())

root
 |-- product_name: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- order_id: integer (nullable = true)
 |-- add_to_cart_order: integer (nullable = true)
 |-- reordered: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- eval_set: string (nullable = true)
 |-- order_number: integer (nullable = true)
 |-- order_dow: integer (nullable = true)
 |-- order_hour_of_day: integer (nullable = true)
 |-- days_since_prior_order: double (nullable = true)
 |-- aisle_id: integer (nullable = true)
 |-- department_id: integer (nullable = true)
 |-- Frq_sold: long (nullable = false)

None


In [67]:
df_merged.columns

['product_name',
 'product_id',
 'order_id',
 'add_to_cart_order',
 'reordered',
 'user_id',
 'eval_set',
 'order_number',
 'order_dow',
 'order_hour_of_day',
 'days_since_prior_order',
 'aisle_id',
 'department_id',
 'Frq_sold']

In [70]:
df = df_merged[["order_id","product_name"]]
df.show(5)

+--------+--------------------+
|order_id|        product_name|
+--------+--------------------+
|      28|Organic Hass Avocado|
|      53|Organic Hass Avocado|
|      81|Organic Hass Avocado|
|     126|Organic Hass Avocado|
|     148|Organic Hass Avocado|
+--------+--------------------+
only showing top 5 rows



In [73]:
basket = df.groupBy("order_id").agg(collect_list(col("product_name")).alias("list_of_values"))

In [74]:
basket.show()

+--------+--------------------+
|order_id|      list_of_values|
+--------+--------------------+
|      28|[Organic Hass Avo...|
|      31|[Organic Yellow O...|
|      53|[Organic Hass Avo...|
|      78|      [Strawberries]|
|      81|[Organic Hass Avo...|
|      85|      [Strawberries]|
|     101|       [Large Lemon]|
|     126|[Organic Hass Avo...|
|     148|[Organic Baby Spi...|
|     193|[Bag of Organic B...|
|     210|       [Large Lemon]|
|     211|      [Strawberries]|
|     251|[Bag of Organic B...|
|     255|            [Banana]|
|     321|[Organic Whole Mi...|
|     451|     [Banana, Limes]|
|     458|[Organic Yellow O...|
|     472|[Organic Whole Mi...|
|     516|            [Banana]|
|     588|[Banana, Organic ...|
+--------+--------------------+
only showing top 20 rows



In [78]:
basket.count()

1642302

In [98]:
df = basket.toPandas()
df.info()

C:\spark/spark-3.3.2-bin-hadoop2\python\pyspark\sql\pandas\utils.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(pandas.__version__) < LooseVersion(minimum_pandas_version):
C:\spark/spark-3.3.2-bin-hadoop2\python\pyspark\sql\pandas\utils.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(pandas.__version__) < LooseVersion(minimum_pandas_version):


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1642302 entries, 0 to 1642301
Data columns (total 2 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   order_id        1642302 non-null  int32 
 1   list_of_values  1642302 non-null  object
dtypes: int32(1), object(1)
memory usage: 18.8+ MB


In [88]:
from pyspark.ml.fpm import FPGrowth
from pyspark.sql.functions import col

# Assuming df_transactions is your DataFrame with columns: transaction_id, items
df_transactions = basket

# FPGrowth requires a DataFrame with a column of arrays containing items
# Assuming you have a column 'items' that contains arrays of transaction items
df_items = df_transactions.select("list_of_values")

# Configure and train the FPGrowth model
fp_growth = FPGrowth(itemsCol="list_of_values",minConfidence=0.01 , minSupport=0.01, predictionCol='prediction')
model = fp_growth.fit(df_items)


In [89]:

# Generate frequent itemsets
frequent_itemsets = model.freqItemsets

# Generate association rules
association_rules = model.associationRules

# Show frequent itemsets and association rules
frequent_itemsets.show()
association_rules.show()

+--------------------+------+
|               items|  freq|
+--------------------+------+
|    [Organic Garlic]|109778|
|[Organic Garlic, ...| 22073|
|[Organic Garlic, ...| 17697|
|[Organic Garlic, ...| 21318|
|[Organic Garlic, ...| 20996|
|[Organic Garlic, ...| 19107|
|            [Banana]|472565|
|  [Organic Zucchini]|104823|
|[Organic Zucchini...| 16464|
|[Organic Zucchini...| 22632|
|[Organic Zucchini...| 20415|
|[Organic Zucchini...| 19318|
|[Bag of Organic B...|379450|
|[Organic Strawber...|264683|
|[Organic Strawber...| 61628|
|[Organic Strawber...| 56156|
|[Organic Blueberr...|100060|
|[Organic Blueberr...| 23756|
|[Organic Blueberr...| 21868|
|[Organic Blueberr...| 18982|
+--------------------+------+
only showing top 20 rows

+--------------------+--------------------+--------------------+------------------+--------------------+
|          antecedent|          consequent|          confidence|              lift|             support|
+--------------------+--------------------+-

In [95]:
rules_pandas = association_rules.toPandas()
rules_pandas.head(5)

C:\spark/spark-3.3.2-bin-hadoop2\python\pyspark\sql\pandas\utils.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(pandas.__version__) < LooseVersion(minimum_pandas_version):
C:\spark/spark-3.3.2-bin-hadoop2\python\pyspark\sql\pandas\utils.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(pandas.__version__) < LooseVersion(minimum_pandas_version):


,antecedent,consequent,confidence,lift,support
0,[Organic Yellow Onion],[Organic Garlic],0.194603,2.911297,0.013440
1,[Organic Yellow Onion],[Organic Strawberries],0.146801,0.910866,0.010139
2,[Organic Yellow Onion],[Organic Hass Avocado],0.175683,1.350870,0.012134
3,[Organic Yellow Onion],[Bag of Organic Bananas],0.216652,0.937695,0.014963
4,[Organic Yellow Onion],[Organic Baby Spinach],0.173743,1.179471,0.012000


In [97]:
# printing Association Rules
for index, rule in rules_pandas.iterrows():
    antecedents = ', '.join(rule['antecedent'])
    consequents = ', '.join(rule['consequent'])
    support = rule['support']
    confidence = rule['confidence']
    lift = rule['lift']
    
    print(f"Rule #{index+1}:")
    print(f"Antecedents: {antecedents}")
    print(f"Consequents: {consequents}")
    print(f"Support: {support}")
    print(f"Confidence: {confidence}")
    print(f"Lift: {lift}")
    print("-----------------------------")

Rule #1:
Antecedents: Organic Yellow Onion
Consequents: Organic Garlic
Support: 0.013440280776617212
Confidence: 0.1946026484227602
Lift: 2.9112966050574425
-----------------------------
Rule #2:
Antecedents: Organic Yellow Onion
Consequents: Organic Strawberries
Support: 0.01013881734297346
Confidence: 0.14680055719147286
Lift: 0.9108663898953475
-----------------------------
Rule #3:
Antecedents: Organic Yellow Onion
Consequents: Organic Hass Avocado
Support: 0.012133578355259873
Confidence: 0.17568282404387
Lift: 1.3508701648667307
-----------------------------
Rule #4:
Antecedents: Organic Yellow Onion
Consequents: Bag of Organic Bananas
Support: 0.014963143197779702
Confidence: 0.21665226667607074
Lift: 0.9376952190450503
-----------------------------
Rule #5:
Antecedents: Organic Yellow Onion
Consequents: Organic Baby Spinach
Support: 0.01199962004552147
Confidence: 0.1737432334738067
Lift: 1.179471231602464
-----------------------------
Rule #6:
Antecedents: Organic Yellow Onion